In [12]:
import serial
import struct
import time

##两个imu分开写
state1 = 0
payload1 = list()

state2 = 0
payload2 = list()

# +SPPDATA=1,028,
header1 = bytes([0x2B, 0x53, 0x50, 0x50, 0x44, 0x41, 0x54, 0x41, 0x3D, 0x31, 0x2C, 0x30, 0x32, 0x38, 0x2C])
# +SPPDATA=2,028,
header2= bytes([0x2B, 0x53, 0x50, 0x50, 0x44, 0x41, 0x54, 0x41, 0x3D, 0x32, 0x2C, 0x30, 0x32, 0x38, 0x2C])

class DataFrame():
    def __init__(self, buffer):
        self.device_addr = buffer[0]
        self.data = buffer[1:25]
        self.invariant = buffer[25:28]
        self.reserved = buffer[28:32]

    def __repr__(self):
        vals = struct.unpack("<6f", self.data)
        vals_str = "{:8.4f},{:8.4f},{:8.4f},{:8.4f},{:8.4f},{:8.4f}".format(vals[0], vals[1], vals[2], vals[3], vals[4], vals[5])
        #invariant_str = "{:02X} {:02X} {:02X}".format(self.invariant[0], self.invariant[1], self.invariant[2])
        #reserved_str = "{:02X} {:02X} {:02X} {:02X}".format(self.reserved[0], self.reserved[1], self.reserved[2], self.reserved[3])
        return vals_str

def getframe1(b):
    global state1
    global payload1

    b = int.from_bytes(b, byteorder='big')
    if state1 == -1 and b == 0x2B:
        state1 = 0
        return False
    elif state1 >= 0 and state1 < 14:
        if header1[state1 + 1] == b:
            state1 = state1 + 1
        else:
            state1 = -1
        return False
    elif state1 >= 14 and state1 < 45:
        payload1.append(b)
        state1 = state1 + 1
        return False
    elif state1 == 45:
        payload1.append(b)
        state1 = -1
        return True

def getframe2(b):
    global state2
    global payload2

    b = int.from_bytes(b, byteorder='big')
    if state2 == -1 and b == 0x2B:
        state2 = 0
        return False
    elif state2 >= 0 and state2 < 14:
        if header2[state2 + 1] == b:
            state2 = state2 + 1
        else:
            state2 = -1
        return False
    elif state2 >= 14 and state2 < 45:
        payload2.append(b)
        state2 = state2 + 1
        return False
    elif state2 == 45:
        payload2.append(b)
        state2 = -1
        return True

In [24]:
def getval(cur):
    whole=[]
    res=cur.split(',')
    for item in res:
        whole.append(eval(item))
    return whole

In [25]:
try:
    port_name = "/dev/tty.usbserial-A50285BI"
    port_bps = 921600
    serial_port = serial.Serial(port_name, port_bps, timeout=1)
    print(serial_port)
    serial_port.write("AT+SPPSEND=1,6,view=1\r\n".encode(encoding="ascii"))
    time.sleep(1)
    serial_port.write("AT+SPPSEND=2,6,view=1\r\n".encode(encoding="ascii"))

    print("command sent")
    f1=open('../data/imu1.txt','w')
    f2=open('../data/imu2.txt','w')
    imu1=[]
    imu2=[]
    
    for i in range(102400):
        data=serial_port.read()
        cur=time.time()
        if getframe1(data):
            frame = DataFrame(bytes(payload1))
            frame = getval(str(frame))
            imu1.append((cur,frame))
            payload1 = []
        if getframe2(data):
            frame = DataFrame(bytes(payload2))
            frame = getval(str(frame))
            imu2.append((cur,frame))
            payload2 = []       
    
    f1.write(str(imu1))
    f2.write(str(imu2))
    
    f1.close()
    f2.close()

except Exception as e:
    print(e)

Serial<id=0x7fdfb1804400, open=True>(port='/dev/tty.usbserial-A50285BI', baudrate=921600, bytesize=8, parity='N', stopbits=1, timeout=1, xonxoff=False, rtscts=False, dsrdtr=False)
command sent


### 数据格式转换
1. 差值
2. 计算角速度

需要注意
- 要把两个imu的时间段统一
- 选择合适的帧率

In [55]:
from scipy.interpolate import interp1d

In [56]:
f1=open('../data/imu1.txt').read()
f2=open('../data/imu2.txt').read()

In [62]:
#看一下两个数据点之间的时间戳
imu1=eval(f1)
tot=0
interval=0
for i in range(1,len(imu1)):
    tot+=1
    interval+=imu1[i][0]-imu1[i-1][0]
    

In [63]:
interval/tot

0.01618837666910401

In [64]:
len(imu1)

1854

In [65]:
len(imu2)

2627

In [67]:
imu1[:10]

[(1625472245.924957, [71.9774, 18.1563, -15.7351, 0.5672, 0.0817, 0.006]),
 (1625472245.925916, [72.6049, 19.8604, -12.9963, 0.4697, 0.038, -0.1919]),
 (1625472245.9267688, [72.7159, 19.893, -10.6339, 0.2624, 0.0794, -0.3742]),
 (1625472245.927407, [72.5912, 19.2017, -9.1508, 0.1319, 0.0854, -0.4289]),
 (1625472245.928041, [72.3306, 18.3538, -8.8933, 0.0853, 0.0465, -0.3546]),
 (1625472245.928693, [72.1455, 17.4403, -9.7528, 0.0817, 0.0684, -0.2213]),
 (1625472245.929397, [72.0754, 16.4494, -11.2768, 0.0949, 0.1086, -0.0438]),
 (1625472245.930042, [71.8189, 15.4645, -13.1491, 0.0854, 0.1222, 0.0487]),
 (1625472245.930639, [71.0508, 14.7431, -15.3202, 0.0539, 0.0959, 0.0877]),
 (1625472245.931196, [70.0649, 14.1517, -17.4385, 0.0132, 0.0211, 0.1])]

In [68]:
from scipy import interpolate
import math

def get_data(path):
    with open(path) as f:
        data=eval(f.read())
    return data

def store(data,path):
    with open(path,'w') as f:
        f.write(str(data))

def get_inter(data,begin,end,diff):
    tot=begin
    whole=[]
    nx=[]
    while tot<end:
        whole.append([tot])
        nx.append(tot)
        tot+=diff
        pass
    x=[]
    for item in data:
        x.append(item[0])
    
    for i in range(len(data[0][1])):
        #对于每个数据点
        cur=[]
        for j in range(len(data)):
            cur.append(data[j][1][i])
        f=interpolate.interp1d(x,cur,kind='linear')
        y=f(nx)
        for j in range(len(whole)):
            whole[j].append(y[j])
    return whole


def trans(path,diff=0.1):
    diff=0.1
    data=[]
    for item in path:
        data.append(get_data(item))
    begin=max(data[0][0][0],data[1][0][0])
    begin=math.floor(begin)
    end=min(data[0][-1][0],data[1][-1][0])
    end=math.ceil(end)
   
    for i in range(len(data)):
        cur=get_inter(data[i],begin,end,diff)
        curpath=path[i].replace('data','newdata',1)
        store(cur,curpath)


In [71]:
diff = 0.1
data = []
path=['../data/imu1.txt','../data/imu2.txt']

for item in path:
    data.append(get_data(item))
begin = max(data[0][0][0], data[1][0][0])
begin = math.floor(begin)
end = min(data[0][-1][0], data[1][-1][0])
end = math.ceil(end)

# for i in range(len(data)):
#     cur = get_inter(data[i], begin, end, diff)
#     curpath = path[i].replace('data', 'newdata', 1)
#     store(cur, curpath)


In [79]:
tot=begin
whole=[]
nx=[]
diff=0.1
data=data[0]
while tot<end:
    whole.append([tot])
    nx.append(tot)
    tot+=diff
x=[]
for item in data:
    x.append(item[0])


In [80]:
for i in range(len(data[0][1])):
    cur=[]
    for j in range(len(data)):
        cur.append(data[j][1][i])
    f=interpolate.interp1d(x,cur,kind='linear')
    y=f(nx)
    for j in range(len(whole)):
        whole[j].append(y[j])


ValueError: A value in x_new is above the interpolation range.

In [81]:
len(data[0][1])

6

In [82]:
len(data)

1854

In [87]:
x[-1]

1625472275.922019

In [84]:
nx[0]

1625472246

In [86]:
nx[-1]

1625472275.9999714

In [88]:
end

1625472276